# Solving the problem of 3-coloring using CSP

This is a small example to show how you can encode the problem of [3-coloring](https://en.wikipedia.org/wiki/Graph_coloring#Vertex_coloring) into a constraint satisfaction problem and using a CSP solver to solve the problem.

We will use Google's [OR-Tools](https://developers.google.com/optimization/introduction/overview) in Python (see this [example](csp.ipynb) for details on how to set this up).

Let's start by creating a CSP instance.

In [1]:
from ortools.sat.python import cp_model

model = cp_model.CpModel();

## 3-coloring

In the problem of 3-coloring, you are given as input a finite (undirected) [graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) $G = (V,E)$, consisting of a set $V$ of *vertices* (or *nodes*) and a set $E$ of *edges*. Each edge $e \in E$ consists of a set $\{v_1,v_2\}$ of exactly two vertices in $V$.

A *coloring* of a graph $G = (V,E)$ is a function $\mu : V \rightarrow C$ that assigns to each vertex $v \in V$ a color $\mu(v) \in C$. The coloring $\mu$ is called *proper* if for every edge $\{v_1,v_2\} \in E$, the coloring assigns different colors to the two endpoints of the edge—that is, $\mu(v_1) \neq \mu(v_2)$.

In the problem of 3-coloring, the question is to decide if there exists a proper coloring $\mu : V \rightarrow \{1,2,3\}$ of the graph that only uses three colors.

## Encoding the graph and colorings

Take some graph $G = (V,E)$, that comprises an input for the 3-coloring problem.
Suppose that the vertices $V = \{ v_1,\dotsc,v_n \}$ are numbered from $1$ to $n$.

We will encode this graph in Python in the following way.

To explain this, take the following example graph $G = (V,E)$, where $V = \{v_1,v_2,v_3,v_4\}$ and $E = \{ \{v_1,v_2\}, \{v_1,v_3\}, \{v_2,v_3\}, \{v_2,v_4\}, \{v_3,v_4\} \}$. We encode the set of edges as a list of pairs, where each vertex is indicated by its index. 

In [2]:
num_vertices = 4;
edges = [(1,2),(1,3),(2,3),(2,4),(3,4)];

Then, we will have to represent the possible 3-colorings of the graph $G$ using variables. We will do this as follows. For each vertex $v_i$, we will have a variable $x_i$ with domain $\{1,2,3\}$.

In [3]:
vars = dict()
for i in range(1,num_vertices+1):
    vars[i] = model.NewIntVar(1, 3, "x{}".format(i));

The different assignments of the variables $x_1,\dotsc,x_4$ to the domain $\{1,2,3\}$ correspond exactly to all possible 3-colorings of the vertices $v_1,\dotsc,v_4$.

## Ensuring that colorings are proper

To make sure that only truth assignments are allowed that correspond to *proper* 3-colorings, we need to add some constraints.

For each edge $\{ v_{i_1},v_{i_2} \} \in E$, and each color $c$, we will make sure that the vertices $v_{i_1}$ and $v_{i_2}$ do not both get colored with color $c$.
For each edge $\{ v_{i_1},v_{i_2} \} \in E$, we add a constraint that expresses that the value assigned to $x_{i_1}$ is different from the value assigned to $x_{i_2}$.

In [4]:
for (i,j) in edges:
    model.Add(vars[i] != vars[j]);

## Calling the CSP solver and constructing a coloring

All together, the constraints in our CSP instance ensure the following. Every assignment that satisfies the constraints (if such an assignment exists) corresponds to a proper 3-coloring of the graph $G$. Therefore, we can now call the CSP solver to find a solution to the CSP instance, if it exists. And if it exists, we can use it to construct a proper 3-coloring of the graph.

For our simple example, a solution to the CSP instance exists, and we get a proper 3-coloring of the graph.

In [5]:
solver = cp_model.CpSolver();
answer = solver.Solve(model);

if answer == cp_model.FEASIBLE:
    print("The graph is 3-colorable:");
    for i in range(1,num_vertices+1):
        print("- Vertex {} gets color {}".format(i,solver.Value(vars[i])));
else:
    print("The graph is not 3-colorable!");

The graph is 3-colorable:
- Vertex 1 gets color 1
- Vertex 2 gets color 3
- Vertex 3 gets color 2
- Vertex 4 gets color 1
